In [101]:
import time
import os
import subprocess
import shutil
import json
import itertools
import functools
import pathlib

import pandas as pd
import numpy as np

import munkres
import generator
import sklearn.cluster

In [54]:
DB, labels = generator.generate_subspacedata(2000, 100, False, [[5, 4, 1, 0.1],
                                                               [5, 6, 1, 0.1],
                                                               [5, 9, 1, 0.1],
                                                               [5, 9, 1, 0.1],
                                                               [5, 9, 1, 0.1]])
np.savetxt("res/sample6.csv", DB, delimiter=",", fmt='%1.8f')

def persist_ground_truth():
    # generates the corresponding ground_truth.csv file from the labels matrix
    with open("res/ground_truth.csv", mode="w") as output_file:
        cluster_counter = 1
        while np.argwhere(labels == cluster_counter).size > 0:
            dimensions = list(set(np.argwhere(labels == cluster_counter)[:, 1]))
            U = list(set(np.argwhere(labels == cluster_counter)[:, 0]))
            cluster_counter += 1
            output_file.write(str(dimensions) + "-" + str(U) + "\n")
persist_ground_truth()

In [77]:
def subscale(epsilon, minpts):

    # get the start time
    st = time.time()

    # the folder "/out" must be deleted before each subscale run
    dirpath = pathlib.Path('out')
    if dirpath.exists() and dirpath.is_dir():
        shutil.rmtree(dirpath)

    SAMPLEFILE = '6'
    GERMAN_FILE_FORMATTING = 'true'
    SPLITTING_SIZE = '16'
    EVENLY_SIZED_SLICES = 'true'
    DBSCAN = 'false'
    p = subprocess.Popen(['java',
                          '-jar',
                          'SubScaleExtended.jar',
                          SAMPLEFILE,
                          GERMAN_FILE_FORMATTING,
                          str(epsilon),
                          str(minpts),
                          SPLITTING_SIZE,
                          EVENLY_SIZED_SLICES,
                          DBSCAN],
                         stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT)
    for line in p.stdout:
        # fix this when java project persists
        pass
        #print(str(line, "utf-8").replace('\r\n', ''))
    print("Duration: ", time.time() - st, " seconds")

In [92]:
def dbscan(eps):
    """
    Potential subspaces will be searched for clusters using the dbscan algorithm.
    The input subspaces are in a csv format. The clusters found are also written to a csv file.
    The algorithm is density-based and is able to recognize multiple clusters. Noise points are ignored.
    """

    # get the start time
    st = time.time()

    with open("out/results/found_clusters.csv", mode="w") as output_file,\
            os.scandir("out/results/subspaces") as it:

        # Iterate over the csv files.
        for entry in it:
            if os.path.getsize( entry.path) == 0:
                # Check for empty file sometimes generated due to bug in SubScaleExtended.jar
                break
            df = pd.read_csv(entry.path, header=None, delimiter="-")

            # Iterate over all potential subspaces
            for _, row in df.iterrows():
                points_indexes = json.loads(row[1])
                dimensions = json.loads(row[0])
                S = DB[points_indexes][:, dimensions]
                clustering = sklearn.cluster.DBSCAN(eps=eps).fit(S)

                # Converting dbscan result in clusters.csv file-format
                unique_labels = set(clustering.labels_)
                unique_labels.discard(-1)  # -1 stands for noisy samples and is therefore removed
                for k in unique_labels:
                    U = np.array(points_indexes)[clustering.labels_ == k]
                    output_file.write(str(dimensions) + "-" + str(U.tolist()) + "\n")

    print("Duration: ", time.time() - st, " seconds")

In [97]:
def f1(c_res, c_ground):
    try:
        recall = len(c_res.intersection(c_ground)) / len(c_ground)
        precision = len(c_ground.intersection(c_res)) / len(c_res)
        return (2 * recall * precision) / (recall + precision)
    except ZeroDivisionError:
        return 0

def f1_recall(ground, res):
    """

    :param ground:
    :param res:
    :return:
    """
    accumulated_scores = 0
    for c_ground in ground:
        best_score = 0
        for c_res in res:
            best_score = max(f1(c_ground, c_res), best_score)
        accumulated_scores += best_score
    try:
        return accumulated_scores / len(ground)
    except ZeroDivisionError:
        return 0

def rnia(size_union, size_intersection):
    """
    Relative NonIntersecting Area (RNIA)
    Formula: 1 - (size_union - size_intersection)/size_union

    :param size_union: Cardinality of the union between all micro_objects from both ground_truth and found_clusters.
    :param size_intersection: Cardinality of the intersection between all micro_objects from both ground_truth and found_clusters.
    """
    return 1 - (size_union - size_intersection)/size_union

def cluster_error(size_union, ground, res):
    """
    The basic idea of cluster_error (CE) is to find a 1:1 mapping between the ground_truth and found clusters.
    For each mapped pair (Cg , Cr ) the cardinality of their intersecting micro_objects is determined.
    Overall, those 1:1 mappings are chosen that result in the highest total sum over all cardinalities.
    This sum is denoted as D_max.
    The formula to calculate cluster_error is: 1 - (size_union - D_max)/size_union
    """

    # M is the confusion matrix between the found_clusters and ground_truth clusters. The values
    # of the individual entries result from the cardinality of the intersection of the respective clusters.
    M = pd.DataFrame(itertools.product(ground,res)).apply(lambda x: len(set(x[0]).intersection(set(x[1]))), axis=1).to_numpy().reshape(len(ground),len(res))


    # We use the Hungarian method to find a permutation of the cluster labels such
    # that the sum of the diagonal elements of M is maximized.
    indexes = munkres.Munkres().compute(munkres.make_cost_matrix(M))
    D_max = 0
    for row, column in indexes:
        value = M[row][column]
        D_max += value

    return 1 - ((size_union - D_max) / size_union)

def score(metric="f1"):
    with open("out/results/found_clusters.csv", mode="r") as found_clusters_file,\
            open("res/ground_truth.csv", mode="r") as ground_truth_file:

        # A dataframe read from the csv format is converted in such a way
        # that the dimensions and point_indexes are available as a set in the corresponding columns.
        # Each row is a cluster with column 0 as the dimensions and column 1 as the point_indexes.
        df_found_clusters = pd.read_csv(found_clusters_file, header=None, delimiter="-")\
            .apply(lambda s: s.apply(lambda x: (set(json.loads(x)))))
        df_ground_truth = pd.read_csv(ground_truth_file, header=None, delimiter="-")\
            .apply(lambda s: s.apply(lambda x: (set(json.loads(x)))))

        if metric=="f1":
            return f1_recall(df_ground_truth[1], df_found_clusters[1])

        # Clustering with Cartesian product of dimensions and point_indexes as cluster
        # representation. Once for ground_truth, once for found_clusters.
        clusters_cartesian_product_ground = [list(itertools.product(x,y)) for x,y in zip(df_ground_truth[0], df_ground_truth[1])]
        clusters_cartesian_product_res = [list(itertools.product(x,y)) for x,y in zip(df_found_clusters[0], df_found_clusters[1])]

        # Entirety of all micro_objects (cartesian products) of the clustering
        # in the form of a set. Once for ground_truth, once for found_clusters.
        micro_objects_ground = set(functools.reduce(lambda a, b: a+b, clusters_cartesian_product_ground))
        micro_objects_res = set(functools.reduce(lambda a, b: a+b, clusters_cartesian_product_res))

        # U is union between all micro_objects from both ground and res
        U = micro_objects_ground.union(micro_objects_res)

        if metric=="rnia":
            # I is intersection between all micro_objects from both ground and res
            I = micro_objects_ground.intersection(micro_objects_res)
            return rnia(len(U), len(I))

        if metric=="ce":
            return cluster_error(len(U), clusters_cartesian_product_ground, clusters_cartesian_product_res)


In [98]:
subscale(epsilon=0.2, minpts=3)

Duration:  3.1206486225128174  seconds


In [99]:
dbscan(0.5)

Duration:  0.05401730537414551  seconds


In [100]:
score("ce")

0.44385026737967914